In [419]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [420]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca[zł/h]"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(employment_date), end_date, 1)[0]
            c += 1
        else:
            dismissal_date = None

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         dismissal_date, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [421]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca[zł/h]
0,1,Wanda,Rzeźnik,73011254649,Wrocław,ul. Gęsia 48,48102088113,wanda.rzeznik@kuba.pl,2014-01-01,None,menadżer,62.0
1,2,Maria,Zawodna,74052249368,Wrocław,ul. Wypoczynkowa 78/24,48145556416,maria.zawodna@kuba.pl,2014-01-01,2016-08-09,sprzedawca,57.0
2,3,Krystyna,Woźnica,68022332323,Wrocław,ul. Urocza 89/6,48097683001,krystyna.woznica@kuba.pl,2014-01-01,None,sprzedawca,47.8
3,4,Mirosław,Świerc,75120243972,Wrocław,ul. Przęsłowa 57/46,48629335383,miroslaw.swierc@kuba.pl,2014-01-01,None,mechanik,46.6
11,5,Piotr,Halushka,83042812457,Ciechanów,ul. Sokoła 34/4,48602712923,piotr.halushka@kuba.pl,2014-01-01,None,informatyk,42.1
4,6,Zdzisław,Błaszczak,94020519879,Chrzanów,ul. Stefana Okrzei 40/12,48332253213,zdzislaw.blaszczak@kuba.pl,2016-05-10,None,mechanik,47.3
5,7,Monika,Grzybowska,67013184761,Wrocław,ul. Kościelna 97/41,48274117297,monika.grzybowska@kuba.pl,2016-10-27,None,mechanik,58.4
6,8,Barbara,Świniarska,72021074829,Wrocław,ul. Sokołowska 28/38,48066212055,barbara.swiniarska@kuba.pl,2016-12-04,2017-01-21,mechanik,59.5
7,9,Tadeusz,Różycki,72040938971,Prudnik,ul. Gorajowicka 82,48292972808,tadeusz.rozycki@kuba.pl,2016-12-17,None,mechanik,53.8
8,10,Anna,Magda,71010886328,Wrocław,Aleja Róż 80/48,48897938705,anna.magda@kuba.pl,2016-12-27,None,pomocnik mechanika,44.3


In [422]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [423]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [424]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [425]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [426]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [427]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [428]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [429]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [430]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [431]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = "M"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = "F"
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]

In [432]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Grażyna,Sukhomlyn,F,78043077718,Gdańsk,ul. Sikorcza 93/7,False,48319400344,sukho_grazyna@outlook.com
1,2,Piotr,Dudek,M,95031464534,Wrocław,ul. Leszczynowa 90,False,48511591645,lh3nm2oc8i5z60f@opayq.com
2,3,Weronika,Liashenko,F,91030847716,Wrocław,ul. Srebrna 66,False,48546886906,w_liashenko1991@vp.pl
3,4,Marta,Abłażewicz,F,73051136673,Zakopane,ul. Wyzwolenia 4/48,True,48427812099,m-ablazewicz@wp.pl
4,5,Jan,Nieśmiałek,M,81072593337,Wrocław,ul. Rakoszycka 23/33,False,48640309961,niesmialek8@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1514,1515,Emil,Rymsza,M,None,Łódź,ul. Henryka Sienkiewicza 58/30,False,48940166377,e-rymsz@onet.pl
1515,1516,Zofia,Miszczyk,F,97122256015,Kędzierzyn-Koźle,ul. Motławska 53/2,False,48821131725,zof.mis@gmail.com
1516,1517,Oleh,Łubiarz,M,96072982311,Grodków,ul. Stroma 19,False,48474446096,olubiarz6577@wp.pl
1517,1518,Kewin,Kubiak,M,71013012157,Lublin,ul. Jarzębinowa 6/45,False,48959938902,kew_k1971@o2.pl


In [433]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [434]:
np.sum(clients_df['karta_lojalnościowa'])

64

In [435]:
np.sum(pd.isna(clients_df["PESEL"]))

60

# auta

In [436]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [437]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [438]:
df_samochody = create_car_table(max(car_ids))

In [439]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,BMW,118,4,2007,Kombi,False,brązowy,Diesel,2.3,143,Manualna,5,7,False
1,2,Skoda,Citigo,4,2006,Coupe,False,biały,Benzyna,1.0,75,Manualna,3,5,False
2,3,smart,forTwo,4,2009,SUV,False,czerwony,Elektryczny,1.5,90,Manualna,5,5,False
3,4,Volkswagen,T-Roc,4,2013,Kombi,False,brązowy,Benzyna,2.3,150,Automatyczna,5,5,False
4,5,BMW,430,4,2008,Kombi,False,czerwony,Diesel,3.9,252,Automatyczna,5,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,969,Renault,Espace,4,2007,SUV,False,czarny,Diesel,2.6,160,Automatyczna,5,5,True
969,970,Hero,Honda Splendor Plus,2,2014,,False,czarny,Benzyna,0.4,55,Manualna,0,1,True
970,971,Ford,Mondeo,4,2009,SUV,False,niebieski,Benzyna,2.6,160,Manualna,5,5,False
971,972,Ford,Mondeo,4,2012,Hatchback,False,szary,Diesel,1.8,120,Manualna,5,5,True


In [440]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

# Usługi

In [441]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksiegowania'] = transaction_dates_service

# Zakup samochodów

In [442]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod)) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [443]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksiegowania'] = transaction_dates_buy

In [444]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksiegowania'] = service_date_fix['data_zaksiegowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksiegowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksiegowania'] = buy_date_fix['data_zaksiegowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksiegowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_17124\1855781291.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2004' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [445]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [446]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,1,1,3,2014-01-03,45900.0
1,2,2,2,2,2014-01-03,22400.0
2,3,3,3,3,2014-01-04,36400.0
3,4,4,4,3,2014-01-04,146700.0
4,5,5,5,2,2014-01-05,95700.0
...,...,...,...,...,...,...
943,944,969,1517,2,2020-12-17,44700.0
944,945,970,1280,2,2020-12-18,23800.0
945,946,972,456,2,2020-12-30,36900.0
946,947,973,1519,2,2020-12-31,29300.0


In [447]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [448]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksiegowania'] = transaction_dates_sell

In [449]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania
0,1,7,21,2,2014-01-25
1,2,20,22,2,2014-01-25
2,3,17,23,3,2014-01-26
3,4,3,24,3,2014-01-28
4,5,14,25,2,2014-01-29
...,...,...,...,...,...
936,937,964,1514,2,2020-12-15
937,938,46,1516,2,2020-12-15
938,939,962,1335,2,2020-12-16
939,940,961,787,2,2020-12-19


In [450]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    print(b_price)
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

df_sell['cena'] = sell_prices

57000.0
86300.0
32500.0
36400.0
14800.0
146700.0
27600.0
70500.0
20200.0
95700.0
61600.0
22400.0
123600.0
45700.0
45900.0
36500.0
29200.0
34800.0
19100.0
83800.0
109100.0
35800.0
30400.0
27700.0
44500.0
19800.0
66200.0
125300.0
59700.0
44600.0
134000.0
32100.0
36900.0
52700.0
22400.0
23500.0
283700.0
23000.0
25900.0
20400.0
50500.0
52500.0
57300.0
57500.0
114400.0
69000.0
49300.0
61900.0
17900.0
16800.0
20400.0
82000.0
33700.0
19100.0
26000.0
20400.0
43500.0
129300.0
59900.0
16000.0
68300.0
15100.0
56800.0
30100.0
55300.0
30800.0
120300.0
41800.0
48300.0
46400.0
21300.0
45600.0
60600.0
88300.0
25900.0
43400.0
20100.0
28800.0
52200.0
59800.0
37800.0
26600.0
31100.0
51600.0
17200.0
28800.0
18100.0
51400.0
53600.0
55400.0
76600.0
18900.0
34900.0
70700.0
27700.0
27000.0
17200.0
53800.0
61400.0
42600.0
52800.0
23200.0
29500.0
35800.0
44100.0
66800.0
18500.0
150600.0
17400.0
22100.0
33600.0
26300.0
69500.0
32700.0
37200.0
65400.0
49700.0
36600.0
42100.0
30200.0
48300.0
29000.0
28700.0
99800.

In [451]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,7,21,2,2014-01-25,82700.0
1,2,20,22,2,2014-01-25,110700.0
2,3,17,23,3,2014-01-26,45600.0
3,4,3,24,3,2014-01-28,50200.0
4,5,14,25,2,2014-01-29,22200.0
...,...,...,...,...,...,...
936,937,964,1514,2,2020-12-15,27600.0
937,938,46,1516,2,2020-12-15,66500.0
938,939,962,1335,2,2020-12-16,11300.0
939,940,961,787,2,2020-12-19,24100.0


In [452]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

In [453]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [454]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(23, array([44], dtype=int64)), (27, array([15], dtype=int64)), (38, array([48], dtype=int64)), (43, array([0], dtype=int64)), (58, array([46,  4], dtype=int64)), (68, array([52, 39], dtype=int64)), (70, array([49,  6, 43], dtype=int64)), (79, array([ 8,  2, 16], dtype=int64)), (93, array([55, 53], dtype=int64)), (94, array([13], dtype=int64)), (106, array([47], dtype=int64)), (109, array([17], dtype=int64)), (128, array([16,  2], dtype=int64)), (142, array([31], dtype=int64)), (147, array([1], dtype=int64)), (149, array([0], dtype=int64)), (151, array([18, 41], dtype=int64)), (174, array([12, 34], dtype=int64)), (177, array([24], dtype=int64)), (200, array([21, 24], dtype=int64)), (203, array([37], dtype=int64)), (210, array([29], dtype=int64)), (218, array([62, 34], dtype=int64)), (224, array([35], dtype=int64)), (226, array([49, 15], dtype=int64)), (236, array([1, 7], dtype=int64)), (245, array([17, 13], dtype=int64)), (247, array([42], dtype=int64)), (254, array([27, 27], dtype=in

In [455]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [456]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Kolektor wydechowy,,500,1,1
1,Radio,Renault Scenic,101,2,1
2,Ogniwo paliwowe,Ford Fiesta,213,3,1
3,Maska,Opel Meriva czarny,391,4,1
4,Tłumik,,200,5,1
...,...,...,...,...,...
159,Rozrząd,,2500,110,1
160,Skrzynia biegów,SEAT Mii Manualna,1196,110,1
161,Lakier,czerwony,131,110,1
162,Światła tylne,Ford Mondeo,84,111,1


# Wyposażenie

In [457]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [458]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Kolektor wydechowy,,500.0
1,2,Radio,Renault Scenic,101.0
2,3,Ogniwo paliwowe,Ford Fiesta,213.0
3,4,Maska,Opel Meriva czarny,391.0
4,5,Tłumik,,200.0
...,...,...,...,...
158,138,Wał korbowy,SEAT Mii,526.0
160,139,Skrzynia biegów,SEAT Mii Manualna,1196.0
161,140,Lakier,czerwony,131.0
162,141,Światła tylne,Ford Mondeo,84.0


In [459]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi - uzupełnienie

In [460]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [461]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksiegowania,cena
0,1,23,45,4,2014-03-01,570
1,2,27,52,4,2014-03-15,110
2,3,38,71,6,2014-03-30,240
3,4,43,81,7,2014-04-11,440
4,5,58,109,8,2014-05-16,880
...,...,...,...,...,...,...
106,107,952,1489,9,2020-11-14,510
107,108,955,1499,4,2020-11-20,2230
108,109,956,653,4,2020-11-21,600
109,110,968,1515,7,2020-12-15,4330


In [462]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [463]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [464]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,2,2,1
2,3,3,3,1
3,4,4,4,1
4,5,5,5,1
...,...,...,...,...
159,160,110,77,1
160,161,110,139,1
161,162,110,140,1
162,163,111,141,1


In [465]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [466]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksiegowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'data_zakupu', 'ilość', 'data_zaksięgowania'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))
    service_date = get_service_date(part_id)

    ##### DO DODANIA
    # generowanie daty zakupu części na podstawie purchase_date
    # generowanie daty zaksięgowania transakcji na podstawie purchase_date
    #####
    
    # row = [i+1, part_id, total_cost, purchase_date, quantity, posting_date]
    # parts_purchase_pd.loc[i] = row


In [467]:
# parts_purchase_pd